### Learning Objectives
* To be able to define what a BST and search for, insert and delete elements from it.
* To be able to identify and fix AVL property violations. 
* To use function pointers to traverse a BST.

### Instructions
Read and study the following sections, run their code examples and solve their challenges. This worksheet has the following challenges:
* [CHALLENGE 01](#ch01)
* [CHALLENGE 02](#ch02)
* [CHALLENGE 03](#ch03)
* [CHALLENGE 04](#ch04)

Run your coding challenges and fix any errors they might have before downloading and submitting your completed worksheet for grading. When done, open the menu **File >> Download as >> HTML (.html)** to download your worksheet in HTML format. **Submit the downloaded *.html* file via Canvas**.

## Function pointers
Before we talk about the main topic of this worksheet, we'll digress a bit to cover **function pointers** in C++, which we will need for our implementation of binary search trees.

A function pointer is as the name states a pointer to a function. That is, it points to the starting position of the function code. Because it points to code and not data, it cannot be allocated or deallocated using **new** or **delete**. It's a special pointer for special situations. 

Say for example, you have the following two functions:

In [ ]:
#include <iostream>
#include <string>

void hello(std::string name, int times){
    for(int i = 0; i < times; i++){
        std::cout << "Hello " << name << "!";
    }
}

In [ ]:
void welcome(std::string name, int times){
    for(int i = 0; i < times; i++){
        std::cout << "Welcome " << name << "!";
    }
}

The statement:

In [ ]:
void (*greet)(std::string, int);

defines a function pointer named `greet` that can point to any function that returns `void` and takes two arguments: a string and an integer. Because both `hello` and `welcome` functions have matching prototypes (return type (`void`) and argument list (a string and an integer)), the function pointer `greet` can be used to point to any of them. For example, let's point `greet` to function `welcome`.

In [ ]:
greet = welcome;

As you can see, the function name `welcome` is used to get the address of its function. Now we can use `greet` to call the `welcome` function.

In [ ]:
greet("John", 3);

We can re-point `greet` to function `hello` and then use it to say `Hello` to `John` three times.

In [ ]:
greet = hello;
greet("John", 3);

We can also pass function pointers as arguments to functions. Let's see a more useful example. 

In the sorting worksheet, we have the following implementation of insertion sort:

```c++
template<typename T>
void insertionSort(std::vector<T>& data) {
    for(int i = 1; i < data.size(); i++){
        for(int j = i; j > 0 && data[j] < data[j - 1]; j--){
            std::swap(data[j], data[j-1]);
        }
    }
}

```
which can only sort the data vector ascendingly. Let's use a function pointer to refactor this function so that it supports both ascending and descending sorting. To start, we create the following comparison functions.

In [ ]:
template<typename T>
bool lessThan(T a, T b){ return a < b; }

In [ ]:
template<typename T>
bool greaterThan(T a, T b) { return a > b; }

Now we implement insertion sort without hardcoding the `<` operator. 

In [ ]:
#include <vector>
template<typename T>
void insertionSort(std::vector<T>& data, bool(*compare)(T,T) = lessThan) {
    for(int i = 1; i < data.size(); i++){
        for(int j = i; j > 0 && compare(data[j], data[j - 1]); j--){
            std::swap(data[j], data[j-1]);
        }
    }
}

This new implementation uses a function pointer named `compare` with a prototype matching that of `lessThan` and `greaterThan`. This function pointer is passed as an argument to `insertionSort` and points to the `lessThan` function by default. Let's see how we can use this new implementation to sort the following data.

In [ ]:
std::vector<int> my_data = { 8, 7, 9, 3, 0, 4, 9, 10, 13, 5 };
insertionSort(my_data);
for(int d : my_data){ std::cout << d << " "; }

And to sort the same data descendingly, we run:

In [ ]:
insertionSort(my_data, greaterThan);
for(int d : my_data){ std::cout << d << " "; }

We now get back to our main topic.


# Binary search trees
A binary search tree (or BST for short) is a data structure that organizes data into a tree-like structure. Such a structure provides an efficient way not only for inserting elements into a tree, but also for searching them or deleting them. In other words, a BST achieves all the benefits of a linked list with a few extra benefits. It makes it possible to search data in $O(log(n))$ time which was only possible with binary search for arrays, not linked lists.

Typically, the data elements that are stored in a BST are unique; **no two elements are have the same value**. Think about a dictionary; you only want to have a single entry per word.

In a BST, every data element is a node. Being binary means that a node can have at most two children. If a node does not have children, it's called a **leaf**. Let's see an example:

```
          6
       /    \
     /        \
    3          8
  /   \      /   \
 /     \    /     \
1       4  7       9
```

In this tree, we call the element at the top (6) the **root**. It has two children: **left** and **right**. The left child of 6 is 3 and the right child of 6 is 8. Similarly, node 3 has two children: 1 (left) and 4(right). The same goes for node 8 with its children 7(left) and 9(right). The nodes 1, 4, 7, and 9 are leaves; they have no children. All nodes, except for the **root**, have a parent. For example, node 8 is the parent of both 7 and 9.

Being a binary search tree (BST) means having the following property: **every node is greater than all the nodes to its left and is less than all the nodes to its right.**. We call this the **binary search property** and it must be maintained for every node in the BST. For example, node 6 is greater that all the nodes to its left (3, 1, and 4) and is less than all the nodes to its right (8, 7, and 9).

With this property, the order in which elements are inserted into the BST controls the shape of the tree. Say for example we have the following data and and we enter them into the BST in the order from left to right.
```
12, 8, 16, 2, 17, 3, 9, 1
```

We start with an empty BST. When we insert 12, it becomes the root of the tree (and the one and only node in it at this time). The tree will look like this:
```
                          12
```

Now we insert 8. Because 8 is less than 12, it will be inserted as the left child of 12. The tree changes to:
```
                          12
                         /
                        8
```
Then we insert 16 which is greater than 12. This makes 16 the right child of 12. The tree becomes:
```
                          12
                         /  \
                        8    16
```
Entering 2, leads to a tree like:
```
                          12
                         /  \
                        8    16
                       /
                      2
```
Then we enter 17:
```
                          12
                         /  \
                        8    16
                       /       \
                      2         17
```
Then we enter 3:
```
                          12
                         /  \
                        8    16
                       /       \
                      2         17
                       \
                        3
```
Then we enter 9:
```
                          12
                         /   \
                        8     16
                       / \      \
                      2   9      17
                       \
                        3
```
And finally we enter 1:

```
                          12
                         /   \
                        8     16
                       / \      \
                      2   9      17
                     / \
                    1   3
```
As you can see the binary tree property is maintained throughout the whole tree. 


Now we define a few properties of the tree. The **size of the tree** is how many nodes it has. The above BST has a size of 8. The **height of the tree** is the number of the nodes in the longest path connecting the root to a leave. We have two longest paths: 12-8-2-1 and 12-8-2-3. They both have 4 nodes which is the height of the tree.

The height of the tree is important, it determines the running time required to insert, delete, or search for an element in the tree.

### <a id="ch01">CHALLENGE 01</a>
**Q1.** Draw the BST that results from inserting the data `12, 8, 16, 2, 17, 3, 9, 1` in the order from right to left.

**Q2.** Draw the BST that results from inserting the data `a, b, c, d, e, f` in an ascending order (a to f)

## Implementing BST
To implement a BST, we need two things:
* a structure representing what a node looks like, and
* a pointer to the root node of the tree

Here is the node structure. 

In [ ]:
template <typename T>
struct Node {
  T info;
  Node *left;
  Node *right;
  Node(T info) : info(info), left(nullptr), right(nullptr) {}
};

Every node has two pointers (left and right) connecting it to its children. We can now use the `Node` structure to create a `BST` class which looks like this:

In [ ]:
enum class Traversal { PRE_ORDER, IN_ORDER, POST_ORDER };

In [ ]:
template <typename T>
class BST {
private:
  Node<T> *root;

public:
  BST(): root(nullptr) {}
  BST(const BST<T> &bt) = delete;
  BST<T> &operator=(const BST<T> &bt) = delete;

  bool empty() const { return !root; }
  int size() const { return size(root); }
  
  void traverse(Traversal type, void (*show_node)(const Node<T> *)) const {
    traverse(type, root, show_node);
  }

  bool search(T e) const; //TODO
  void insert(T e); //TODO
  void remove(T e); //TODO

  ~BST(){ destroy(root); }

private:
  void destroy(Node<T> *node){
    if (node) {
      destroy(node->left);
      destroy(node->right);

      delete node;
      node = nullptr;
    }
  }
    
  int size(const Node<T>* node); //TODO
  void traverse(Traversal type, const Node<T> *node, 
                void (*show_node)(const Node<T> *)) const; //TODO
};


We now implement the functions marked with `//TODO`. 

## BST size
As stated above, the size of a BST is the number of nodes it has. We can implement `size` recursively like this:

In [ ]:
template <typename T>
int BST<T>::size(const Node<T>* node){
    if(node){
        return 1 + size(node->left) + size(node->right);
    }else{
        return 0;
    }
}

That is the size of any tree whose root is `node` is 1 plus the sum of its left tree and right tree sizes.

## BST traversal
Traversing a BST tree means visiting and reporting every node in it. There are three kinds of traversals based on when the node is reported out:
- **Pre-order traversal:** where node is reported out before its left and right subtrees.
- **In-order traversal:** where node is reported out after its left subtree and before its right subtree.
- **Post-order traversal:** where node is reported out after its left and right subtrees.

And we use a function pointer (`show_node`) to avoid hardcoding how a node is reported out. Here is our recursive traversal implementation.

In [ ]:
template <typename T>
void BST<T>::traverse(Traversal type, const Node<T> *node, 
                      void (*show_node)(const Node<T> *)) const{
    if (node) {
      switch(type){
      case Traversal::PRE_ORDER:
        show_node(node);
        traverse(type, node->left, show_node);
        traverse(type, node->right, show_node);
        break;
      case Traversal::IN_ORDER:
        traverse(type, node->left, show_node);
        show_node(node);
        traverse(type, node->right, show_node);
        break;
      case Traversal::POST_ORDER:
        traverse(type, node->left, show_node);
        traverse(type, node->right, show_node);
        show_node(node);
        break;  
      }
    }
  }


## Searching BST
Searching a BST for a value `e` involves starting at the root node and and comparing that value to its info. If it's less than the root, we follow the left link, else we follow the right link. We keep doing this until the value is found or the bottom of the tree is reached. Here is how the implementation looks:

In [ ]:
template <typename T>
bool BST<T>::search(T e) const {
    auto current = root;
    while(current){
      if(current->info == e){
        return true;
      }else if(e < current->info){
        current = current->left;
      }else if(e > current->info){
        current = current->right;
      }
    }

    return false;
}  

## Inserting an element into a BST
Early on in this worksheet, we saw an example of constructing a BST by inserting elements into it one at a time. We start at the root and compare the new value to its info. if it's less than the root we go left, else we go right. The goal is find the node that is going to be the parent of the new value. When the search is done, we compare the new value to its to-be parent node, if it's less than it, we insert it as a left child; if it's greater than it we insert it as a right child. Here is the implementation for that. Notice that duplicate values are not allowed in this implementation.

In [ ]:
template <typename T>
void BST<T>::insert(T e){
    if(this->root){
      auto current = this->root;
      Node<T>* parent = nullptr;
      while(current){
        parent = current;
        if(current->info == e){
          throw std::runtime_error("element is already in the list");
        }else if(e < current->info){
          current = current->left;
        }else if(e > current->info){
          current = current->right;
        }
      }

      if(e < parent->info){
        parent->left = new Node<T>(e);
      }else{
        parent->right = new Node<T>(e);
      }
    } else{
      this->root = new Node<T>(e);
    }
}

## Removing elements from a BST
To remove an element `e` from a BST, we first search for that element and if it is not found, we return and do nothing. If it is found, however, we have a few cases to consider. Let's go over these cases with examples one by one.

Say that we have this tree:

```
                          12
                         /   \
                        8     14
                       / \      \
                      2   9      17
                     / \        /  \    
                    1   3      15   19
```
**CASE 1**: Removing a node that has no children. Let's remove node 1 which has no children, what do we get? Well! We get the following tree:

```
                          12
                         /   \
                        8     14
                       / \      \
                      2   9      17
                       \        /  \    
                        3      15   19
```
We only had to delete the node 1 and make the left link of 2 null. The same goes if we were to delete 9; we delete it and make the right link of 8 null, which results in a tree like this:
```
                          12
                         /   \
                        8     14
                       /        \
                      2          17
                       \        /  \    
                        3      15   19
```
**CASE 2**: Removing a node with a left child only. For example, removing node 8 which has a left child 2, results in a tree like this:

```
                          12
                         /   \
                        2     14
                         \      \
                          3      17
                                /  \    
                               15   19                
```
Here we link 12 (8's parent) via a left link to 8's only left child(2) and then delete 8. If the to-be-deleted node is linked to its parent with a right link, then we link the parent to the only child of the to-be-deleted node via a right link.

**CASE 3**: Removing a node with right child only. This is similar to **CASE 2**. For example, removing node 14 which has a right child 17, results in a tree like this:

```
                          12
                        /    \
                       2     17
                        \    /  \
                         3  15   19               
```
Here we link 12 (14's parent) via a right link to 14's only right child(17) and then delete 14. If the to-be-deleted node is linked to its parent with a left link, then we link the parent to the only child of the to-be-deleted node via a left link.

**CASE 4**: Removing a node with two children. Say that we want to delete a node `d` that has two children. First we go to the left size of node `d` and look for a node `m` with the maximum value of all nodes to the left of `d`. Having found `m`. We then exchange d's value with m's value and then delete the node m. For example, let's delete the node 17 which has two children 15 and 19. So d = 17 and m = 15 (the maximum value to the left of 17). We then move the value 15 to where 17 is and remove its leaf node, which results in a tree like this:

```
                          12
                        /    \
                       2     15
                        \      \
                         3     19               
```
What about deleting the root node 12? If d = 12, then m = 3 (the maximum value to its left). That means getting the tree

```
                          3
                        /   \
                       2     15
                              \
                              19               
```
Here is the implementation of the remove function.

In [ ]:
template <typename T>
void BST<T>::remove(T e){
    // Search of the node to be deleted
    auto node = this->root;
    Node<T>* parent = nullptr;
    bool is_left_child = false;
    while(node){
      if(node->info == e){
        break;
      }else {
        parent = node;
        if(e < node->info){
          is_left_child = true;
          node = node->left;
        }else if(e > node->info){
          is_left_child = false;
          node = node->right;
        }
      }
    }

    if(node){
      // CASE 1: Node has not children
      if(!node->left && !node->right) {
        if(is_left_child){
          parent->left = nullptr;
        }else{
          parent->right = nullptr;
        }
      }else if(node->left && !node->right){
        // CASE 2: Node has only a left child
        if(is_left_child){
          parent->left = node->left;
        }else{
          parent->right = node->left;
        }
      }else if(!node->left && node->right){
        // CASE 3: Node has only a right child
        if(is_left_child){
          parent->left = node->right;
        }else{
          parent->right = node->right;
        }
      }else{
        // CASE 4: Node has both left and right children
        // First find the node with the maximum value on the left side
        auto current = node->left;
        Node<T>* beforeCurrent = nullptr;
        while(current->right){
          beforeCurrent = current;
          current = current->right;
        }

        node->info = current->info;

        if(beforeCurrent){
          beforeCurrent->right = current->left;
        }else{
          node->left = current->left;
        }

        // Node to delete
        node = current;
      }
      
      delete node;
      node = nullptr;
    }
}

## Testing our implementation
To test our implementation, we first need a function to print the node during tree traversal. Here is a simple function to do that. 

In [ ]:
template <typename T>
void simplePrint(const Node<T> *node){
    if(node){
      std::cout << node->info << " ";
    }
}

Let's test our BST class.

In [ ]:
BST<int> bst;
std::vector<int> data = { 12, 8, 16, 2, 17, 3, 9, 1 };
for(int d : data){ bst.insert(d); }

bst.traverse(Traversal::IN_ORDER, simplePrint);

### <a id="ch02">CHALLENGE 02</a>
**Q1**. Draw the BST that results from inserting the elements `{ 12, 8, 16, 2, 17, 3, 9, 1 }` from left to right.

**Q2**. What is the size of this BST?

**Q2**. What is the height of this BST?

**Q3**. How many leaves (leaf nodes) does it have?

### <a id="ch03">CHALLENGE 03</a>
**Q1**. Starting with the BST of CHALLENGE 02, draw this BST after inserting 7.

**Q2**. Draw this BST after deleting 2. Which CASE was that?

**Q3**. What will the height of this BST be after deleting 1?

## Analyzing search, insert, and remove
Looking at the search, insert, and remove functions, we see that there three functions, travel down the tree starting at the root. In the worst case, they go as far as the deepest leaf. Than means the running time for these algorithms is $O(h)$ where $h$ is the height of the BST. But what is $h$ in terms of $n$: the size of the tree? Well, it depends on whether you have a balanced BST or not. A unbalanced BST has most nodes in one side (left or right) and a few or none on the other side. Think about the following BST:
```
  5
   \
    6
     \
      7
       \
        8
```
With an extremely unbalance tree like this, $h = n$, which means that searching, inserting, and removing will happen at linear time $O(n)$. If the tree is balanced (with similar(not necessarily equal) number of nodes on both sides), the we get a BST like this:
```
                          12
                         /   \
                        8     14
                       / \      \
                      2   9      17
                       \        /  \    
                        3      15   19
```
where $h \approx log(n)$ which means that searching, inserting, and removing will happen at close to logarithmic time $O(log(n))$.

The question is then, how do make sure that our BST's are fairly balanced? Well we adopt and implement a *self-balancing* strategy. We have two such strategies: **AVL trees**, and **Red-Black trees**. The videos of this course and the in-class project will cover AVL trees.

### <a id="ch04">CHALLENGE 04</a>
In a BST created by inserting the following elements from left to right:
```
G, E, H, B, A, F, H, I, K, O, J
```
**Q1**. What will pre-order traversal print out? Separate values by space.

**Q2**. What will in-order traversal print out? Separate values by space.

**Q3**. What will post-order traversal print out? Separate values by space.

# AVL Trees
An AVL tree is a self-balancing binary search tree that guarantees the height of the tree is approximately $log(n)$. The AVL tree attaches a height value to every node. The height of a node $x$ is the maximum number of nodes on the path from that node ($x$) to a leaf, including $x$ itself. And the height of the tree is the height of its root node. For example, here is an AVL tree, which as you see is a fairly balanced BST. 

```
                          12
                         /   \
                        8     14
                       / \    /  \
                      2   9  13   17
                       \         /  \    
                        3       15   19
```
Here are the heights of all the nodes in this tree:

| Node  | Height |
|-------|--------|
| 12    | 4      |
| 8     | 3      |
| 14    | 3      |
| 2     | 2      |
| 9     | 1      |
| 13    | 1      |
| 17    | 2      |
| 3     | 1      |
| 15    | 1      |
| 19    | 1      |


The **AVL tree** self-balances by enforcing a **balancing condition** making sure that any node insertion or removal does not violate this condition. The balancing condition used by the AVL tree is the following:

    At any given node, the difference in heights between the node's left child and the node's right child cannot be more than one. 
    
We call this the **AVL property** and it must be maintained on all nodes. Take a moment to inspect the AVL tree above and to make sure that this property is not violated anywhere in the tree.

Inserting new elements into the tree or removing existing nodes from the tree can result in a violation of this property. For example, inserting 25, results in BST like this:

```
                          12
                         /   \
                        8     14
                       / \    /  \
                      2   9  13   17
                       \         /  \    
                        3       15   19
                                      \
                                       25
```
This is not an AVL tree because the AVL property is violated at node 14. To see why that is the case, we calculate the height of node 14's left child (node 13) which is 1 and the height of its right child (node 17) which is 3. The  difference between these two heights $|1 - 3|$ is more than 1. To fix this we left-rotate node 14 (where the violation happens) as described in the lecture videos. This results in an AVL tree like this:

```
                           12
                         /     \
                        8       17
                       / \     /  \
                      2   9   14   19
                       \     /  \    \    
                        3   13   15   25
```

Removing node 9 will result in a BST that violates the AVL property like this:

```
                           12
                         /     \
                        8       17
                       /       /  \
                      2       14   19
                       \     /  \    \    
                        3   13   15   25
```
The violation happens at node 8 because its left child has a height of 2 and its right child has height of 0 (no right child). The difference between 3 and 0 is more than one. We fix this by performing two (double) rotations. First we left-rotate at node 2 which results int the following BST tree (which is not yet an AVL tree):
```
                           12
                         /     \
                        8       17
                       /       /  \
                      3       14   19
                     /        /  \    \    
                    2       13   15   25
```
Then we right rotate at node 8, gives the following AVL tree.
```
                           12
                         /     \
                        3       17
                       / \     /  \
                      2   8   14   19
                              /  \    \    
                           13   15   25
```

The details of the rotations needed to fix AVL property violations were discussed in the lecture videos. Here is a step-by-step summary of that process: 

- Starting from the bottom-up, find the node $x$ where the violation of the AVL property happens.
- Find the node $y$ by finding the the child of $x$ with the larger height. if $x$'s left child has the larger height, then `y = x.left` else `y = x.right`.
- Find which child of $y$ has the larger height.
- Use the following rotation table.

| $x$'s child with larger height | $y$'s child with larger height | Action               |
|--------------------------------|--------------------------------|----------------------|
| right                          | right                          | left-rotate at $x$   |
| right                          | left                           | right-rotate at $y$<br>left-rotate at $x$   |
| left                           | left                           | right-rotate at $x$   |
| left                           | right                          | left-rotate at $y$<br>right-rotate at $x$   |

# B-trees
A B-tree is an efficient self-balancing search tree. It generalizes the notion of search trees to nodes with more than one key and more than two children. A b-tree of order $m$ is called an $m$-way b-tree and has the following properties:
* All leaves are at the same level.
* Non-leaf and non-root internal nodes have at least $\lceil\frac{m}{2}\rceil$ children and at most $m$ children.
* Leaves and non-root internal nodes have at least $\lceil\frac{m}{2}\rceil - 1$ keys and at most $m - 1$ keys.
* The root node has at least $2$ children and at most $m$ children. It has at least $1$ key and at most $m - 1$ keys.

For example, for the common 5-way b-tree we have:
* $m = 5$
* $m - 1 = 4$
* $\lceil\frac{m}{2}\rceil = \lceil\frac{5}{2}\rceil = \lceil2.5\rceil = 3$
* For non-leaf and non-root internal nodes, $ 3 \leq \#\ of\ children \leq 5$
* For leaves and non-root internal nodes, $ 2 \leq \#\ of\ keys \leq 4$
* For the root node, $ 2 \leq \#\ of\ children \leq 5$ and $ 1 \leq \#\ of\ keys \leq 5$.

Implementing B-trees is beyond the scope of this course. To get an idea on what this implementation would look like, here is a structure representing a node in a B-tree.

In [ ]:
template<typename T, int order>
struct BNode {
    int n; // Number of actual keys in node
    T keys[order - 1];
    BNode* children[order];
};

which as you can see is a generalization of the `Node<>` structure used for BST.